# Load the Pretrained Model and the dataset
We use ernie-1.0 as the model and chnsenticorp as the dataset for example. More models can be found in [PaddleNLP Model Zoo](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/transformers.html#transformer).

Obviously, PaddleNLP is needed to run this notebook, which is easy to install:
```bash
pip install setuptools_scm 
pip install --upgrade paddlenlp==2.1 
```

In [1]:
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

MODEL_NAME = "ernie-1.0"

model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

[2021-11-04 17:44:32,722] [    INFO] - Already cached /root/.paddlenlp/models/ernie-1.0/ernie_v1_chn_base.pdparams
W1104 17:44:32.724829 23696 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.2
W1104 17:44:32.730230 23696 device_context.cc:422] device: 0, cuDNN Version: 7.6.
[2021-11-04 17:44:37,381] [    INFO] - Already cached /root/.paddlenlp/models/ernie-1.0/vocab.txt


In [2]:
from paddlenlp.datasets import load_dataset
DATASET_NAME = 'chnsenticorp'
train_ds, dev_ds, test_ds = load_dataset(
    DATASET_NAME, splits=["train", "dev", "test"]
)

# Prepare the Model

## Train the model

In [3]:
# training the model and save to save_dir
# only needs to run once.
# total steps ~900 (3 epochs)

from assets.utils import training_model
training_model(model, tokenizer, train_ds, dev_ds, save_dir=f'assets/{DATASET_NAME}-{MODEL_NAME}')

# global step 900, epoch: 3, batch: 300, loss: 0.00739, acc: 0.98438
# eval loss: 0.19582, accu: 0.94750

dataset labels: ['0', '1']
dataset examples:
{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'qid': ''}
{'text': '15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错', 'label': 1, 'qid': ''}
{'text': '房间太小。其他的都一般。。。。。。。。。', 'label': 0, 'qid': ''}
{'text': '1.接电源没有几分钟,电源适配器热的不行. 2.摄像头用不起来. 3.机盖的钢琴漆，手不能摸，一摸一个印. 4.硬盘分区不好办.', 'label': 0, 'qid': ''}
{'text': '今天才知道这书还有第6卷,真有点郁闷:为什么同一套书有两种版本呢?当当网是不是该跟出版社商量商量,单独出个第6卷,让我们的孩子不会有所遗憾。', 'label': 1, 'qid': ''}
Training Starts:
global step 100, epoch: 1, batch: 100, loss: 0.11187, acc: 0.78969
global step 200, epoch: 1, batch: 200, loss: 0.42704, acc: 0.84484
global step 300, epoch: 1, batch: 300, loss: 0.08328, acc: 0.87125
eval loss: 0.18355, accu: 0.93417
global step 400, epoch: 2, batch: 100, loss: 0.16292, acc: 0.96375
global step 500, epoch: 2, batch: 200, loss: 0.06864, acc: 0.96188
global step 600, epoch: 2, batch: 300, loss: 0.15862, acc: 0.96073
eval l

## Or Load the trained model

In [3]:
# Load the trained model.
state_dict = paddle.load(f'assets/{DATASET_NAME}-{MODEL_NAME}/model_state.pdparams')
model.set_dict(state_dict)

# See the prediction results

In [4]:
from assets.utils import predict

data = [
    {"text":'这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'},
    {"text":'怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'},
    {"text":'作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'},
]

label_map = {0: 'negative', 1: 'positive'}

batch_size = 32

results = predict(
    model, data, tokenizer, label_map, batch_size=batch_size)

for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))

Data: {'text': '这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般'} 	 Lable: negative
Data: {'text': '怀着十分激动的心情放映，可是看着看着发现，在放映完毕后，出现一集米老鼠的动画片'} 	 Lable: negative
Data: {'text': '作为老的四星酒店，房间依然很整洁，相当不错。机场接机服务很好，可以在车上办理入住手续，节省时间。'} 	 Lable: positive


# Prepare for Interpretations

In [5]:
import interpretdl as it
import numpy as np
from assets.utils import convert_example, aggregate_subwords_and_importances
from paddlenlp.data import Stack, Tuple, Pad
from interpretdl.data_processor.visualizer import VisualizationTextRecord, visualize_text

def preprocess_fn(data):
    examples = []
    
    if not isinstance(data, list):
        data = [data]
    
    for text in data:
        input_ids, segment_ids = convert_example(
            text,
            tokenizer,
            max_seq_length=128,
            is_test=True
        )
        examples.append((input_ids, segment_ids))

    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input id
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # segment id
    ): fn(samples)
    
    input_ids, segment_ids = batchify_fn(examples)
    return paddle.to_tensor(input_ids, stop_gradient=False), paddle.to_tensor(segment_ids, stop_gradient=False)

InterpretDL Current Version: 0.3.0rc


## IG Interpreter

In [10]:
ig = it.IntGradNLPInterpreter(model, True)

pred_labels, pred_probs, avg_gradients = ig.interpret(
    preprocess_fn(data),
    steps=100,
    return_pred=True)

true_labels = [1, 1, 0, 0] * 5
recs = []
for i in range(avg_gradients.shape[0]):
    subwords = " ".join(tokenizer._tokenize(data[i]['text'])).split(' ')
    subword_importances = avg_gradients[i]
    words, word_importances = aggregate_subwords_and_importances(subwords, subword_importances)
    word_importances = np.array(word_importances) / np.linalg.norm(
        word_importances)
    
    pred_label = pred_labels[i]
    pred_prob = pred_probs[i, pred_label]
    true_label = true_labels[i]
    interp_class = pred_label
    
    if interp_class == 0:
        word_importances = -word_importances
    recs.append(
        VisualizationTextRecord(words, word_importances, true_label,
                                pred_label, pred_prob, interp_class)
    )

visualize_text(recs)

## LIME Interpreter

In [13]:
true_labels = [0, 0, 1]
recs = []

lime = it.LIMENLPInterpreter(model)
for i, review in enumerate(data):
    pred_class, pred_prob, lime_weights = lime.interpret(
        review,
        preprocess_fn,
        num_samples=1000,
        batch_size=32,
        unk_id=tokenizer.convert_tokens_to_ids('[UNK]'),
        pad_id=tokenizer.convert_tokens_to_ids('[PAD]'),
        return_pred=True)

    # subwords
    subwords = " ".join(tokenizer._tokenize(review['text'])).split(' ')
    interp_class = list(lime_weights.keys())[0]
    weights = lime_weights[interp_class][1 : -1]
    subword_importances = [t[1] for t in lime_weights[interp_class][1 : -1]]
    
    words, word_importances = aggregate_subwords_and_importances(subwords, subword_importances)
    word_importances = np.array(word_importances) / np.linalg.norm(
        word_importances)
    
    true_label = true_labels[i]
    
    if interp_class == 0:
        word_importances = -word_importances
        
    rec = VisualizationTextRecord(
        words, 
        word_importances, 
        true_label,                   
        pred_class[0], 
        pred_prob[0],
        interp_class
    )
    
    recs.append(rec)

visualize_text(recs)

## GradShapNLPInterpreter

In [15]:
ig = it.GradShapNLPInterpreter(model, True)

pred_labels, pred_probs, avg_gradients = ig.interpret(
    preprocess_fn(data),
    n_samples=50,
    noise_amount=0.1,
    return_pred=True)

true_labels = [0, 0, 1]
recs = []
for i in range(avg_gradients.shape[0]):
    subwords = " ".join(tokenizer._tokenize(data[i]['text'])).split(' ')
    subword_importances = avg_gradients[i]
    words, word_importances = subwords, subword_importances
    word_importances = np.array(word_importances) / np.linalg.norm(
        word_importances)
    
    pred_label = pred_labels[i]
    pred_prob = pred_probs[i, pred_label]
    true_label = true_labels[i]
    interp_class = pred_label
    
    if interp_class == 0:
        word_importances = -word_importances
    recs.append(
        VisualizationTextRecord(words, word_importances, true_label,
                                pred_label, pred_prob, interp_class)
    )

visualize_text(recs)